In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from LaneInput import LaneInput  # Class to input lane regions
from PSMorg import SpeedEstimator2  # Updated SpeedEstimator2 class with time increment
from time_utils import increment_time, get_time_from_xml  # Import utility functions

# Initialize the YOLO model for object detection
model = YOLO(r'E:\learn phython\speed 2\Speed-detection-of-vehicles\best(3).pt')  # Replace with the path to your YOLO model
names = model.model.names  # List of class names for the YOLO model

# Initialize video capture
video_path = r'E:\learn phython\speed 2\Speed-detection-of-vehicles\C0148.MP4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Error reading video file {video_path}"

# Get video properties (width, height, and FPS)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, 
                                      cv2.CAP_PROP_FRAME_HEIGHT, 
                                      cv2.CAP_PROP_FPS))

# Video writer to save the processed video with lane detection
output_video = "lane_detection_output.avi"
video_writer = cv2.VideoWriter(output_video, 
                               cv2.VideoWriter_fourcc(*"mp4v"), 
                               fps, (w, h))

# Initialize LaneInput to capture lane points
lane_input = LaneInput()

# Capture the first frame from the video to input lane points
success, first_frame = cap.read()
if not success:
    raise ValueError("Failed to read the first frame from the video.")

# Input the lane regions in the first frame
lane_input.input_lanes(first_frame)
lane_regions = lane_input.get_lanes()  # Get the captured lane regions
print("Captured Lane Regions:", lane_regions)

# Load initial time from the XML file
xml_file = r"E:\learn phython\speed 2\Speed-detection-of-vehicles\C0148M01.XML"
current_time = get_time_from_xml(xml_file)

# Initialize SpeedEstimator2
lane_estimator = SpeedEstimator2(
    names=names,
    lane_regions=lane_regions,
    view_img=True,  # Display image flag
    target_x=500,   # Define the X-coordinate target for lane estimation
     csv_file="lane_log.csv" # Define the CSV file for logging
    
)

# Frame processing
frame_counter = 0
skip_frames = 10  # Number of frames to skip after processing one frame

# Main loop for processing each video frame
while cap.isOpened():
    success, im0 = cap.read()  # Read the next frame
    if not success:
        print("Video processing completed or frame not available.")
        break

    # Process every (skip_frames + 1)-th frame
    if frame_counter % (skip_frames + 1) == 0:
        # Perform object detection and tracking using the YOLO model
        tracks = model.track(im0, persist=True)
        
        # Increment the current time based on the number of frames skipped
        current_time = increment_time(current_time, skip_frames + 1, fps)
        
        # Pass the tracks and the frame to SpeedEstimator2's check_and_log_lanes function
        im0 = lane_estimator.check_and_log_lanes(im0, tracks, current_time)

        # Display the processed frame with lane estimation
        cv2.imshow("Live Output", im0)

        # Write the processed frame to the output video
        video_writer.write(im0)

    # Increment frame counter
    frame_counter += 1

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Processing terminated by user.")
        break

# Release video capture and writer resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processing complete. Output saved to {output_video}")


Please input the number of lanes:
Point selected: (173, 171)
Point selected: (67, 822)
Point selected: (333, 833)
Point selected: (240, 171)
Lane region added: ((173, 171), (67, 822), (333, 833), (240, 171))
Point selected: (240, 171)
Point selected: (336, 831)
Point selected: (645, 833)
Point selected: (371, 166)
Lane region added: ((240, 171), (336, 831), (645, 833), (371, 166))
Point selected: (374, 165)
Point selected: (647, 831)
Point selected: (927, 832)
Point selected: (490, 155)
Lane region added: ((374, 165), (647, 831), (927, 832), (490, 155))
Point selected: (493, 154)
Point selected: (937, 833)
Point selected: (1195, 829)
Point selected: (586, 153)
Lane region added: ((493, 154), (937, 833), (1195, 829), (586, 153))
Point selected: (583, 150)
Point selected: (1197, 831)
Point selected: (1508, 712)
Point selected: (654, 151)
Lane region added: ((583, 150), (1197, 831), (1508, 712), (654, 151))
Captured Lane Regions: [((173, 171), (67, 822), (333, 833), (240, 171)), ((240, 17

KeyboardInterrupt: 